<a href="https://colab.research.google.com/github/datagirl98/Required-Assignment-17.1/blob/main/prompt_III.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [78]:
import pandas as pd
import numpy as np
import time
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, roc_auc_score

In [66]:
# Load the drive helper
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [67]:
# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Practical_3/bank-additional-full.csv',delimiter=';')

In [68]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



In [69]:
# 1. Basic structure
print("\n=== Dataset Info ===")
print(df.info())

# 2. Preview dataset
print("\n=== First 5 Rows ===")
print(df.head())

# 3. Missing values
print("\n=== Missing Values per Column ===")
print(df.isnull().sum())

# 4. Columns with > 50% missing values
print("\n=== Columns with >50% Missing ===")
print(df.columns[df.isnull().mean() > 0.5])

# 5. Check for duplicate rows
print("\n=== Number of Duplicate Rows ===")
print(df.duplicated().sum())

# 6. Summary statistics for numeric columns
print("\n=== Summary Statistics (Numerics) ===")
print(df.describe())

# 7. Data types and unique values (categoricals)
print("\n=== Unique Values in Categorical Columns ===")
for col in df.select_dtypes(include='object'):
    print(f"{col}: {df[col].nunique()} unique values")

# 8. Outlier check using Z-score (optional)
from scipy.stats import zscore
z_scores = np.abs(zscore(df.select_dtypes(include=np.number)))
outliers = (z_scores > 3).sum(axis=0)
print("\n=== Outlier Count (Z-Score > 3) ===")
print(outliers)

# 9. Constant columns (same value throughout)
print("\n=== Constant Columns ===")
print([col for col in df.columns if df[col].nunique() == 1])



=== Dataset Info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf

### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

The business objective is to predict a customer's likelyhood to subscribe to a term deposit with this marketing campaign.

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

In [71]:
# Define bank-related features and target
bank_features = [
    'job', 'marital', 'education', 'default', 'housing',
    'loan', 'contact', 'month', 'day_of_week', 'poutcome'
]
target = 'y'

# Separate features and target
X = df[bank_features]
y = df[target].map({'yes': 1, 'no': 0})  # Convert target to binary

# Set up one-hot encoding for categorical features
categorical_transformer = OneHotEncoder(drop='first', sparse_output=False)
# Apply transformation
preprocessor = ColumnTransformer(
    transformers=[('cat', categorical_transformer, bank_features)]
)

# Transform features
X_encoded = preprocessor.fit_transform(X)

# Retrieve encoded column names
encoded_feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(bank_features)

# Convert to DataFrame
X_encoded_df = pd.DataFrame(X_encoded, columns=encoded_feature_names)

# Optional: Show the prepared data
print(X_encoded_df.head())
print(y.head())


   job_blue-collar  job_entrepreneur  job_housemaid  job_management  \
0              0.0               0.0            1.0             0.0   
1              0.0               0.0            0.0             0.0   
2              0.0               0.0            0.0             0.0   
3              0.0               0.0            0.0             0.0   
4              0.0               0.0            0.0             0.0   

   job_retired  job_self-employed  job_services  job_student  job_technician  \
0          0.0                0.0           0.0          0.0             0.0   
1          0.0                0.0           1.0          0.0             0.0   
2          0.0                0.0           1.0          0.0             0.0   
3          0.0                0.0           0.0          0.0             0.0   
4          0.0                0.0           1.0          0.0             0.0   

   job_unemployed  ...  month_may  month_nov  month_oct  month_sep  \
0             0.0  ...

### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [72]:
# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded_df, y, test_size=0.3, random_state=42)

# Train a simple model
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Predict and evaluate
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.96      0.94     10968
           1       0.45      0.24      0.32      1389

    accuracy                           0.88     12357
   macro avg       0.68      0.60      0.63     12357
weighted avg       0.86      0.88      0.87     12357



### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [73]:
# Count class distribution
class_counts = y.value_counts()
print(class_counts)

# Baseline accuracy = proportion of majority class
baseline_accuracy = class_counts.max() / class_counts.sum()
print(f"Baseline accuracy: {baseline_accuracy:.4f}")

y
0    36548
1     4640
Name: count, dtype: int64
Baseline accuracy: 0.8873


### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [74]:
# Train logistic regression model
logreg = LogisticRegression(max_iter=1000, random_state=42)
logreg.fit(X_train, y_train)

# Make predictions
y_pred = logreg.predict(X_test)

# Evaluate performance
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Display results
print(f"Accuracy: {accuracy:.4f}")
print("\nConfusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", report)

Accuracy: 0.8971

Confusion Matrix:
 [[10825   143]
 [ 1128   261]]

Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.99      0.94     10968
           1       0.65      0.19      0.29      1389

    accuracy                           0.90     12357
   macro avg       0.78      0.59      0.62     12357
weighted avg       0.88      0.90      0.87     12357



### Problem 9: Score the Model

What is the accuracy of your model?

The accuracy of the model is: **0.8971**

### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [75]:
# Assuming X_encoded_df and y are already defined
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded_df, y, test_size=0.3, random_state=42, stratify=y
)

# List of models to evaluate
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "KNN": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "SVM": SVC()
}

# Store results
results = []

for name, model in models.items():
    start = time.time()
    model.fit(X_train, y_train)
    end = time.time()

    train_acc = model.score(X_train, y_train)
    test_acc = model.score(X_test, y_test)
    train_time = end - start

    results.append({
        "Model": name,
        "Train Time": round(train_time, 4),
        "Train Accuracy": round(train_acc, 4),
        "Test Accuracy": round(test_acc, 4)
    })

# Create and display DataFrame
results_df = pd.DataFrame(results).sort_values(by="Test Accuracy", ascending=False)
print(results_df)

                 Model  Train Time  Train Accuracy  Test Accuracy
3                  SVM    112.2753          0.8986         0.8990
0  Logistic Regression      0.6229          0.8955         0.8971
1                  KNN      0.0058          0.9051         0.8920
2        Decision Tree      0.2310          0.9524         0.8670


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

Gender is not a feature in the data set, however even if it was it should not be included as it could lead to biased or unethical predictions.

In [76]:
# KNN tuning
knn_params = {'n_neighbors': list(range(3, 21))}
knn_grid = GridSearchCV(KNeighborsClassifier(), knn_params, cv=5)
knn_grid.fit(X_train, y_train)
print("Best K for KNN:", knn_grid.best_params_)

# Decision Tree tuning
dt_params = {'max_depth': [3, 5, 10, 15, 20]}
dt_grid = GridSearchCV(DecisionTreeClassifier(random_state=42), dt_params, cv=5)
dt_grid.fit(X_train, y_train)
print("Best max_depth for Decision Tree:", dt_grid.best_params_)

Best K for KNN: {'n_neighbors': 19}
Best max_depth for Decision Tree: {'max_depth': 5}


In [80]:
# Define models and parameter grids
models_params = {
    'KNN': {
        'model': KNeighborsClassifier(),
        'params': {'model__n_neighbors': list(range(3, 21))}
    },
    'Decision Tree': {
        'model': DecisionTreeClassifier(random_state=42),
        'params': {'model__max_depth': [5, None]}
    },
    'Logistic Regression': {
        'model': LogisticRegression(max_iter=1000, random_state=42, class_weight='balanced'),
        'params': {'model__C': [0.01, 0.1, 1, 10, 100]}
    },
    'SVM': {
        'model': SVC(probability=True),
        'params': {'model__C': [0.1, 1, 10], 'model__kernel': ['linear', 'rbf']}
    }
}

# Store results
tuning_results = []

# Run GridSearchCV for each model
for name, mp in models_params.items():
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('model', mp['model'])
    ])

    grid = GridSearchCV(pipeline, mp['params'], cv=5, scoring='f1', n_jobs=-1)
    grid.fit(X_train, y_train)

    best_model = grid.best_estimator_
    train_acc = best_model.score(X_train, y_train)
    test_acc = best_model.score(X_test, y_test)
    f1 = f1_score(y_test, best_model.predict(X_test))
    roc_auc = roc_auc_score(y_test, best_model.predict_proba(X_test)[:, 1])

    tuning_results.append({
        "Model": name,
        "Best Params": grid.best_params_,
        "Train Accuracy": round(train_acc, 4),
        "Test Accuracy": round(test_acc, 4),
        "F1 Score": round(f1, 4),
        "ROC AUC": round(roc_auc, 4)
    })

# Results DataFrame
tuning_results_df = pd.DataFrame(tuning_results)
print(tuning_results_df.sort_values(by="F1 Score", ascending=False))

KeyboardInterrupt: 

##### Questions